In [102]:
import cv2
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm  # Sử dụng tqdm.notebook cho giao diện đẹp hơn trong notebook
import matplotlib.pyplot as plt

# Tắt cảnh báo không cần thiết
import warnings
warnings.filterwarnings('ignore')
import traceback

In [103]:
# ==== Tham số ====
PROJECT_DIR = Path.cwd().parent 

# Đường dẫn đến thư mục dữ liệu
DATA_DIR = PROJECT_DIR / "data"
INTERIM_DIR = DATA_DIR / "02_interim"
PROCESSED_DIR = DATA_DIR / "03_processed"

# Tạo thư mục đầu ra nếu chưa tồn tại
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Thư mục dữ liệu tạm thời (đầu vào): {INTERIM_DIR}")
print(f"Thư mục dữ liệu đã xử lý (đầu ra): {PROCESSED_DIR}")


Thư mục dữ liệu tạm thời (đầu vào): /Users/CONG HAU/UIT-CS-PROJECTS/CS231 - Introduction to Computer Vision/data/02_interim
Thư mục dữ liệu đã xử lý (đầu ra): /Users/CONG HAU/UIT-CS-PROJECTS/CS231 - Introduction to Computer Vision/data/03_processed


In [104]:
def extract_color_features(image, bins=(8, 8, 8)):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, bins,
                        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [105]:
def extract_edge_features(image, grid_size=(8, 8), canny_thresholds=(100, 200)):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, canny_thresholds[0], canny_thresholds[1])
    h, w = edges.shape
    cell_h, cell_w = h // grid_size[0], w // grid_size[1]
    
    edge_density_features = []
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            cell = edges[i*cell_h:(i+1)*cell_h, j*cell_w:(j+1)*cell_w]
            density = np.sum(cell > 0) / (cell.size + 1e-6) 
            edge_density_features.append(density)
    return np.array(edge_density_features)


In [106]:
def extract_feature_sets(image):
    """
    Trích xuất và trả về một dictionary chứa các bộ đặc trưng khác nhau từ một ảnh.
    Hàm này sẽ xử lý ảnh đầu vào có kiểu dữ liệu float [0, 1].
    """
    # === THÊM BƯỚC SỬA LỖI Ở ĐÂY ===
    # Chuyển đổi ảnh từ float (dải 0-1) về uint8 (dải 0-255) mà OpenCV cần
    if image.dtype == np.float32 or image.dtype == np.float64:
        image_uint8 = (image * 255).astype(np.uint8)
    else:
        image_uint8 = image.copy()
    # ================================

    # Đảm bảo ảnh có kích thước cố định
    resized_image = cv2.resize(image_uint8, (128, 128))

    # Trích xuất từng loại đặc trưng từ ảnh đã chuyển đổi
    color_feat = extract_color_features(resized_image)
    edge_feat = extract_edge_features(resized_image)
    
    return {
        "color": color_feat,
        "edge": edge_feat
    }

In [107]:
def process_and_save_features(input_npz_path, output_npz_path):
    if not input_npz_path.exists():
        print(f"Cảnh báo: Không tìm thấy file {input_npz_path}. Bỏ qua.")
        return

    print(f"⏳ Bắt đầu xử lý: {input_npz_path.name}")
    
    # Tải dữ liệu ảnh và nhãn
    data = np.load(input_npz_path)
    images = data['X']
    labels = data['y']
    
    # Thu thập các dictionary đặc trưng từ mỗi ảnh
    all_features_list = []
    for img in tqdm(images, desc=f"Trích xuất từ {input_npz_path.name}"):
        try:
            features_dict = extract_feature_sets(img)
            all_features_list.append(features_dict)
        except Exception as e:
            print(f"Lỗi khi xử lý một ảnh. Chi tiết:")
            traceback.print_exc()
            

    # Nhóm các đặc trưng cùng loại lại với nhau
    if not all_features_list:
        print("Không có đặc trưng nào được trích xuất. Dừng lại.")
        return
        
    feature_keys = all_features_list[0].keys()
    feature_arrays = {
        key: np.array([d[key] for d in all_features_list])
        for key in feature_keys
    }

    # Lưu tất cả các mảng đặc trưng và nhãn vào một file .npz duy nhất
    np.savez_compressed(
        output_npz_path,
        color=feature_arrays["color"],
        edge=feature_arrays["edge"],
        labels=labels
    )
    
    print(f"✅ Đã lưu các bộ đặc trưng tại: {output_npz_path}")
    print(f"   -> Đặc trưng màu (color): {feature_arrays['color'].shape}")
    print(f"   -> Đặc trưng cạnh (edge): {feature_arrays['edge'].shape}")
    print(f"   -> Nhãn (labels): {labels.shape}\n")

In [108]:
# Chạy Quá trình Trích xuất

# --- Xử lý tập Train ---
train_input_path = INTERIM_DIR / "train_data.npz"
train_output_path = PROCESSED_DIR / "train_data.npz"
process_and_save_features(train_input_path, train_output_path)

# --- Xử lý tập Validation ---
val_input_path = INTERIM_DIR / "val_data.npz"
val_output_path = PROCESSED_DIR / "val_data.npz"
process_and_save_features(val_input_path, val_output_path)

# --- Xử lý tập Test ---
test_input_path = INTERIM_DIR / "test_data.npz"
test_output_path = PROCESSED_DIR / "test_data.npz"
process_and_save_features(test_input_path, test_output_path)

print("🎉 Hoàn tất quá trình trích xuất và lưu đặc trưng!")

⏳ Bắt đầu xử lý: train_data.npz


Trích xuất từ train_data.npz:   0%|          | 0/8226 [00:00<?, ?it/s]

✅ Đã lưu các bộ đặc trưng tại: /Users/CONG HAU/UIT-CS-PROJECTS/CS231 - Introduction to Computer Vision/data/03_processed/train_data.npz
   -> Đặc trưng màu (color): (8226, 512)
   -> Đặc trưng cạnh (edge): (8226, 64)
   -> Nhãn (labels): (8226,)

⏳ Bắt đầu xử lý: val_data.npz


Trích xuất từ val_data.npz:   0%|          | 0/2742 [00:00<?, ?it/s]

✅ Đã lưu các bộ đặc trưng tại: /Users/CONG HAU/UIT-CS-PROJECTS/CS231 - Introduction to Computer Vision/data/03_processed/val_data.npz
   -> Đặc trưng màu (color): (2742, 512)
   -> Đặc trưng cạnh (edge): (2742, 64)
   -> Nhãn (labels): (2742,)

⏳ Bắt đầu xử lý: test_data.npz


Trích xuất từ test_data.npz:   0%|          | 0/2743 [00:00<?, ?it/s]

✅ Đã lưu các bộ đặc trưng tại: /Users/CONG HAU/UIT-CS-PROJECTS/CS231 - Introduction to Computer Vision/data/03_processed/test_data.npz
   -> Đặc trưng màu (color): (2743, 512)
   -> Đặc trưng cạnh (edge): (2743, 64)
   -> Nhãn (labels): (2743,)

🎉 Hoàn tất quá trình trích xuất và lưu đặc trưng!


In [ ]:
# Kiểm tra file đã lưu
print("Kiểm tra file train_data.npz...")
try:
    check_data = np.load(train_output_path)
    print("Các key có trong file:", list(check_data.keys()))
    print("Kích thước đặc trưng màu:", check_data['color'].shape)
    print("Kích thước đặc trưng cạnh:", check_data['edge'].shape)
    print("Kích thước nhãn:", check_data['labels'].shape)
    
except FileNotFoundError:
    print("Không tìm thấy file để kiểm tra.")

Kiểm tra file train_data.npz...
Các key có trong file: ['color', 'edge', 'labels']
Kích thước đặc trưng màu: (8226, 512)
Kích thước đặc trưng cạnh: (8226, 64)
Kích thước nhãn: (8226,)
